# **Hi Coders** Today We Are Going To Work Through A Problem Called **Face Emotion Recognition**.
## This is an important topic in field of ML.We Used Our Model to detect human emotion based on their photos.
It Mainly Consist of 7 emotions -->

`Happy` `Sad` `Anger` `Surprise` `Fear` `Neutral` `Disgust`.

Algo Predict HumanEmotions.

I Have Divided This Project in two parts -
1) Creating My Own Model.

2) Using a pretrained Model.

Obviously Second Work Going to work A Lot Better. I Have Not Even Created A Very Deep Learning Neural Network of my own.
Like Follwing a Research paper but if you want to go I Will Suggest To Look at these  **Mollahosseini** and **Lopes**.Because My Semester Exams Are Coming 🥲.
Maybe in Future I Will  Work On `Mollahosseini` CNN  Model .

On A Tough Intution He Did Follwoing Things-->
## 1) Data Preprocessing:
* Facial landmarks were extracted from the images.
* The images were then resized to 48x48 pixels.
* Augmentation techniques were applied to the data. Augmentation can involve operations like rotation, scaling, flipping, etc., to artificially increase the size of the training dataset and improve the model's generalization.

## 2) Network Architecture:
* In Conv Layers They Use Two Max Pooling Layers And And Two Conv2D Layer.
* And After That They Use `Inception-Style Modules`With Filters With 1x1 and 3x3 and 5x5.

### The Dataset we have Taken Is `Facial-Expression-dataset` on Kaggle

### Importing The Neccessary Libraries

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import random
import warnings
import seaborn as sns

from keras.utils import load_img



warnings.filterwarnings("ignore")

In [ ]:
print(tf.__version__)

### Load The Dataset `Train` And `Test`

In [ ]:
TRAIN_DIR="/kaggle/input/facial-expression-dataset/train"
TEST_DIR="/kaggle/input/facial-expression-dataset/test"

In [ ]:
def load_datasets(directory):
    
    image_paths=[]
    labels=[]
    for label in os.listdir(directory):
        for filename in os.listdir(directory+"/"+label):
            for file in os.listdir(directory+"/"+label+"/"+filename):
                image_path=os.path.join(directory,label,filename,file)
                image_paths.append(image_path)
                labels.append(filename)
            print(filename,"Completed\n")
    return image_paths,labels

### Converting Them Into Train Dataframe

In [ ]:
train=pd.DataFrame()
train["image"],train["label"]=load_datasets(TRAIN_DIR)

In [ ]:
train.sample(4)

### Converting Them Into Test Dataframe

In [ ]:
test=pd.DataFrame()
test["image"],test["label"]=load_datasets(TEST_DIR)

In [ ]:
test.sample(4)

In [ ]:
train=train.sample(frac=1).reset_index(drop=True)

In [ ]:
train.head(4)

### Basic `EDA`

In [ ]:
train["label"].value_counts().values

In [ ]:
sns.countplot(x=train["label"])
plt.show()

In [ ]:
from PIL import Image
img=Image.open(train["image"][10])
plt.imshow(img,cmap="gray")
plt.title(train["label"][10])
plt.show()

In [ ]:
plt.figure(figsize=(25,25))
files=train.iloc[0:25]
for index,file,label in files.itertuples():
#     print(index)
    plt.subplot(5,5,index+1)
    img=Image.open(file)
    plt.imshow(img,cmap="gray")
    plt.title(label)
    plt.axis("off")
#     plt.show()
    
    

What below Fuction are doing is just they looping through all my image converting them into gray_scale  and converting them into a numpy array and creating a list which contains all images in an array format and after that converting that list into a numpy array and then reshaping into shapes of total_images,48,48,1 where 48 are height and width and 1 is channel you can use 3 for `rgb` values


In [ ]:
def load_img_grayscale(path):
    img = Image.open(path)
    img = img.convert('L')
    return img


def extract_feature_images(images):
    features = []
    for image in images:
        img = load_img_grayscale(image) 
        img = np.array(img)
        features.append(img)
    features = np.array(features)
    features = features.reshape(len(features), 48, 48, 1)
    return features
        

In [ ]:
train_features=extract_feature_images(train["image"])


In [ ]:
train_features[0][0][0]

In [ ]:
test_features=extract_feature_images(test["image"])
test_features[0][0][0]

we are dividing those all values by 255. 

Basically We Are MinMaxScaling Are Values to be in form 0 and 1.

`0` means `Black` and `1` means `White`

In [ ]:
X_train=train_features/255.0
X_test=test_features/255.0

In [ ]:
X_train[0][0][0]

## Label Encoding
### to convert values like Happy Sad etc. to 0,1 etc.

In [ ]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
le.fit(train["label"])
y_train=le.transform(train["label"])
y_test=le.transform(test["label"])

In [ ]:
y_train

In [ ]:
y_test

In [ ]:
input_shape=(48,48,1)
output_classes=7

Converting all values into categorical column to help our `CNN Layers` 
to `one_hot_encode` them

In [ ]:
from tensorflow.keras.utils import to_categorical

In [ ]:
y_train=to_categorical(y_train,num_classes=7)


In [ ]:
y_train[0]

In [ ]:
y_test=to_categorical(y_test,num_classes=7)

In [ ]:
y_test

# Making Our Deep CNN Model


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D,LSTM,AveragePooling2D

In [ ]:
model = Sequential()


model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(output_classes, activation="softmax"))

model.compile(optimizer="adam", metrics=["accuracy"], loss="categorical_crossentropy")


### Using `Early Stopping` 

In [ ]:
from keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss',  
                               patience=5,         
                               restore_best_weights=True)

In [ ]:
history=model.fit(x=X_train,y=y_train,batch_size=32,epochs=100,validation_data=(X_test,y_test),callbacks=[early_stopping])

### Plotting the accuracy and val_accuracy graph

In [ ]:
plt.plot(history.history["accuracy"],"b",label="accuracy")
plt.plot(history.history["val_accuracy"],"r",label="val_accuracy")
plt.title("Accuracy Score")
plt.show()

### Plotting the loss and val_loss graph

In [ ]:
plt.plot(history.history["loss"],"b",label="loss")
plt.plot(history.history["val_loss"],"r",label="val_loss")
plt.title("Loss")
plt.show()

## Evaluation our model 
Using `Accuracy` `Precision` `Recall` `F1` and `hinge loss`

In [ ]:
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score

In [ ]:
preds=model.predict(X_test)
preds=np.argmax(preds,axis=1)
preds

In [ ]:
preds_cat=to_categorical(preds,num_classes=7)

In [ ]:
preds_cat

In [ ]:
accuracy_score(y_test,preds_cat)

In [ ]:
precision_score(y_test,preds_cat,average="weighted")

In [ ]:
recall_score(y_test,preds_cat,average="weighted")

In [ ]:
f1_score(y_test,preds_cat,average="weighted")

In [ ]:
from sklearn.metrics import hamming_loss

In [ ]:
hamming_loss_val = hamming_loss(y_test,preds_cat)
print("Hamming Loss:", hamming_loss_val)

In [ ]:
len(X_test)

In [ ]:
import random
image_index=random.randint(0,len(X_test))
print("Original Output:", test['label'][image_index])
pred=model.predict(X_test[image_index].reshape(1, 48, 48, 1))
prediction_label = le.inverse_transform([pred.argmax()])[0]
print("Predicted Output:", prediction_label)
plt.imshow(X_test[image_index].reshape(48, 48), cmap='gray');

In [ ]:
image_index=random.randint(0,len(X_test))
print("Original Output:", test['label'][image_index])
pred=model.predict(X_test[image_index].reshape(1, 48, 48, 1))
prediction_label = le.inverse_transform([pred.argmax()])[0]
print("Predicted Output:", prediction_label)
plt.imshow(X_test[image_index].reshape(48, 48), cmap='gray');

In [ ]:
image_index=random.randint(0,len(X_test))
print("Original Output:", test['label'][image_index])
pred=model.predict(X_test[image_index].reshape(1, 48, 48, 1))
prediction_label = le.inverse_transform([pred.argmax()])[0]
print("Predicted Output:", prediction_label)
plt.imshow(X_test[image_index].reshape(48, 48), cmap='gray');

In [ ]:
model.save('CNN_My_Model.h5')


# Using Pretrained Network **Efficient Net**

Efficient Net Mainly follow an approact to improve model accuracy using 3 things mainly `Depth` of a neural network.`Width` of a network i.e.; channels and `Resultion` of An Image


In [ ]:
!pip install efficientnet


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from efficientnet.tfkeras import EfficientNetB0

### Performing Data Augmentation

In [ ]:
train_datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)
test_datagen = ImageDataGenerator()


In [ ]:
X_train_rgb = np.repeat(X_train[..., np.newaxis], 3, -1)
X_test_rgb = np.repeat(X_test[..., np.newaxis], 3, -1)

### Building Model

In [ ]:
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(48, 48, 3))
base_model.trainable = False

pretrained_model = Sequential([
    base_model,
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(7, activation='softmax')
])

In [ ]:
pretrained_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
X_train_rgb = np.squeeze(X_train_rgb, axis=-2)
X_test_rgb = np.squeeze(X_test_rgb, axis=-2)

In [ ]:
batch_size = 32
epochs = 40

train_generator = train_datagen.flow(X_train_rgb, y_train, batch_size=batch_size)
test_generator = test_datagen.flow(X_test_rgb, y_test, batch_size=batch_size)

In [ ]:
pretrained_history = pretrained_model.fit(train_generator, epochs=epochs, validation_data=test_generator)


### Checking Accuracy and loss of Model

In [ ]:
plt.plot(pretrained_history.history["accuracy"],"b",label="accuracy")
plt.plot(pretrained_history.history["val_accuracy"],"r",label="val_accuracy")
plt.title("Accuracy Score")
plt.show()

In [ ]:
plt.plot(pretrained_history.history["loss"],"b",label="loss")
plt.plot(pretrained_history.history["val_loss"],"r",label="val_loss")
plt.title("Loss Score")
plt.show()

# My Model Performed Well on the Data So I Am Going to use it